In [1]:
# Import the data set
from keras.datasets import imdb
# split the data set into training and testing target-data
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data()

In [2]:
# get all the words already tokenized(remember that the words are unique, each having an ID) -> returns a dictionary
index = imdb.get_word_index()
# reverse key-value pair in dictionary
reverse_index = dict([(value, key) for (key, value) in index.items()])

In [ ]:
# decode a sentence (this is sample code to be used later)
decoded = " ".join([reverse_index.get(i -10, "#") for i in training_data[0]] )
print(decoded) 

## N-Grams

In [3]:
# Get top 20 unigrams in the data set
# Steps:
# 1 decode all the sentences
# 2 tokenize
# 3 count how many times each word recurs
# 4 order the result and print the words
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import numpy as np
# STEP 1: decode all the sentences
# extract all the sentences from training data and test data decoded
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

decoded_sentences = []
# extract from training data
for x in range(len(data)):
    decoded_sentences.append(" ".join([reverse_index.get(i - 3, "") for i in data[x]]))


In [ ]:
# STEP 2: Tokenize
from nltk import word_tokenize
from nltk.util import ngrams

# The function will split everything in Unigrams
def extract_unigram(sentences): 

  tokens = []
  for sentence in sentences:
    tok = word_tokenize(sentence)
    for t in tok:
      tokens.append(t)
  return tokens



# The function will split everything in Bigrams
def extract_bigrams(sentences): 

  all_bigrams = []
  for sentence in sentences:
    token = word_tokenize(sentence)
    bigrams = ngrams(token,2)
    for b in bigrams:
      all_bigrams.append(b)
  return all_bigrams

In [ ]:
# Extract top unigrams
extracted_unigrams = extract_unigram(decoded_sentences);

# Get most frequent unigrams into a dictionary
frequent_unigrams = dict()
for x in extracted_unigrams:
    if x in frequent_unigrams:
            frequent_unigrams[x] += 1
    else:
        frequent_unigrams[x] = 1


In [ ]:

reversed_frequent_unigrams = dict([(value, key) for (key, value) in frequent_unigrams.items()])
reversed_frequent_unigrams_top_values = sorted(reversed_frequent_unigrams.keys(), reverse=True)[:20]
#print(reversed_frequent_unigrams_top_values)

# print unigrams top values
for x in reversed_frequent_unigrams_top_values:
    print(reversed_frequent_unigrams[x])

In [ ]:
# Extract top bigrams
extracted_bigrams = extract_bigrams(decoded_sentences);

# Get most frequent bigrams into a dictionary
frequent_bigrams = dict()
for x in extracted_bigrams:
    if x in frequent_bigrams:
            frequent_bigrams[x] += 1
    else:
        frequent_bigrams[x] = 1
        
    
reversed_frequent_bigrams = dict([(value, key) for (key, value) in frequent_bigrams.items()])
reversed_frequent_bigrams_top_values = sorted(reversed_frequent_bigrams.keys(), reverse=True)[:20]
# print(reversed_frequent_bigrams_top_values)

# print unigrams top values
for x in reversed_frequent_bigrams_top_values:
    print(reversed_frequent_bigrams[x])

## BOW

In [5]:
# Extracting BOW
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
#CountVectorizer implements both tokenization and occurrence counting in a single class.

In [6]:

sentences_as_bag_of_words = vectorizer.fit_transform(decoded_sentences)


In [7]:
vectorizer.get_feature_names();

In [8]:
bow_vectors = sentences_as_bag_of_words.toarray()

In [9]:
bow_vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:

labels = pd.DataFrame(targets)
dataset_frame = pd.DataFrame(data=bow_vectors)
dataset_frame['Class'] = labels

In [17]:
# select lessa data
selected_number = 2001
dataset_reduced = dataset_frame[:selected_number]

In [18]:
# decision tree
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 

#Split the data into a training and a testing set
bound_param = 1930
train_features = dataset_reduced.iloc[:bound_param,:-1] 
test_features = dataset_reduced.iloc[bound_param:,:-1] 
train_labels = dataset_reduced.iloc[:bound_param,-1] 
test_labels = dataset_reduced.iloc[bound_param:,-1]

tree = DecisionTreeClassifier(criterion = 'entropy').fit(train_features,train_labels)#The accuracy is then calculated through the score function offered by sklearn
print("The prediction accuracy is: ",tree.score(test_features,test_labels)*100,"%")


The prediction accuracy is:  74.64788732394366 %


array([0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 0], dtype=int64)

In [67]:
# make predictions
prediction = tree.predict(test_features)
prediction

# convert test labels to array to iterate through
labels_dataFrame = pd.DataFrame(test_labels.values)
labels_dataFrame = labels_dataFrame.to_numpy()


correct_count = 0
wrong_count = 0
for count,x in enumerate(prediction):
    if labels_dataFrame[count] == x:
        print(f'Correct, {x}')
        correct_count+=1
    else:
        print(f'Wrong, {x}')
        wrong_count+=1
        

print(f'Correctly predicted: {correct_count} \nWrongly predicted: {wrong_count}')

Correct, 0
Correct, 1
Correct, 0
Wrong, 0
Correct, 1
Wrong, 0
Correct, 0
Correct, 0
Correct, 0
Correct, 1
Correct, 1
Wrong, 0
Correct, 1
Correct, 1
Correct, 1
Correct, 1
Wrong, 0
Wrong, 1
Correct, 1
Correct, 1
Wrong, 0
Correct, 1
Correct, 0
Correct, 0
Correct, 1
Correct, 0
Correct, 0
Wrong, 0
Correct, 1
Correct, 0
Correct, 0
Correct, 1
Correct, 1
Wrong, 1
Correct, 0
Wrong, 0
Correct, 0
Wrong, 1
Correct, 1
Correct, 1
Correct, 0
Correct, 0
Correct, 1
Correct, 1
Correct, 1
Wrong, 0
Correct, 1
Correct, 1
Wrong, 0
Correct, 1
Correct, 0
Wrong, 1
Correct, 1
Wrong, 1
Correct, 1
Correct, 0
Correct, 1
Wrong, 1
Correct, 1
Correct, 0
Correct, 1
Correct, 0
Wrong, 0
Wrong, 1
Correct, 1
Correct, 1
Wrong, 0
Correct, 0
Correct, 0
Correct, 1
Correct, 0
Correctly predicted: 53 
Wrongly predicted: 18


In [19]:
import io
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = io.StringIO()
export_graphviz(tree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())